## **Set covering problem**

build a minimum number of facilities to cover all demands.

 ***MIP Model***

***Indices and Parameters***

- $I = \{1, 2, ..., m\}$: A set of demand points.
- $J = \{1, 2, ..., n\}$: A set of potential facility locations.
- $d_{ij}$: Distance from demand point $i$ to location $j$, for $i \in I, j \in J$, where $d_{ij} > 0$.
- $s$: A given service level, indicating that demand $i$ is said to be covered by location $j$ if $d_{ij} < s$.
- $a_{ij}$: Binary parameter, where $a_{ij} = 1$ if $d_{ij} < s$ (i.e., demand $i$ can be covered by location $j$), or $0$ otherwise.


***Decision Variables***

- $x_j$: Binary variable that equals 1 if a facility is allocated at location $j$ and 0 otherwise, for $j \in J$.



***Objective Function***

Minimize the total number of facilities allocated:

$$ \text{Minimize } Z = \sum_{j \in J} x_j $$

This objective seeks to allocate as few facilities as possible while meeting all coverage requirements.

***Constraints***

Ensure every demand point is covered by at least one facility within the service level $s$, using the binary parameter $a_{ij}$:

$$ \sum_{j \in J} a_{ij} x_j \geq 1, \quad \forall i \in I $$

This constraint guarantees that each demand point $i$ is covered by at least one facility that is within the service level distance $s$ from it.

***Variable Bounds***

The decision variables are binary:

$$ x_j \in \{0,1\}, \quad \forall j \in J $$


***Code***

In [ ]:
import pulp

class SetCoveringProblem:
    def __init__(self, I, J, d, s):
        """
        Initializes the Set Covering Problem model.

        Parameters:
        - I: Set of demand points.
        - J: Set of potential facility locations.
        - d: Dictionary with keys (i, j) representing distances from demand i to location j.
        - s: Service level, a maximum distance for demands to be considered covered.
        """
        self.I = I
        self.J = J
        self.d = d
        self.s = s
        
        # Calculate a_ij based on d_ij and s
        self.a = {(i, j): 1 if d[(i, j)] < s else 0 for i in I for j in J}

        # Initialize the problem
        self.problem = pulp.LpProblem("SetCoveringProblem", pulp.LpMinimize)
        
        # Decision variables
        self.x = pulp.LpVariable.dicts("x", J, cat=pulp.LpBinary)

    def build_model(self):
        """
        Builds the MIP model for the set covering problem.
        """
        # Objective Function
        self.problem += pulp.lpSum(self.x[j] for j in self.J), "Minimize_Total_Facilities"
        
        # Constraints: Ensure every demand point is covered
        for i in self.I:
            self.problem += pulp.lpSum(self.a[(i, j)] * self.x[j] for j in self.J) >= 1, f"Coverage_Constraint_{i}"

    def solve(self):
        """
        Solves the set covering problem and prints the results.
        """
        self.problem.solve()
        print(f"Status: {pulp.LpStatus[self.problem.status]}")
        
        # Print the solution
        for j in self.J:
            if self.x[j].value() == 1:
                print(f"Facility at location {j} is allocated.")



Example:

In [ ]:
# Example usage
I = [1, 2, 3]  # Demand points
J = [1, 2, 3, 4]  # Potential facility locations
d = {(1, 1): 2, (1, 2): 3, (1, 3): 3, (1, 4): 1,
     (2, 1): 4, (2, 2): 2, (2, 3): 1, (2, 4): 3,
     (3, 1): 3, (3, 2): 4, (3, 3): 2, (3, 4): 2}  # Distances
s = 3  # Service level

# Create and solve the model
problem = SetCoveringProblem(I, J, d, s)
problem.build_model()
problem.solve()